In [1]:
%cd ../

/u02/thanhhm/research-idea


In [2]:
import plasma.parallel_processing as pp
import time

In [3]:
def compute(data):
    time.sleep(1)
    return data * 2

def print_pass(data):
    print(f'{data / 2**5}')
    return data

In [4]:
class Outer(pp.communicators.Flow):

    def __init__(self):
        super().__init__()
        manager = pp.mp.Manager()

        self.compute1 = Inner()
        self.compute2 = compute
        self.compute3 = Inner()
        self.printer = print_pass
        self.aggregator = pp.communicators.Aggregator(20, 0.5, manager)

        self.registerIOs(
            compute1={
                'compute1': pp.queues.ProcessQueue(2),
                'compute2': pp.queues.ProcessQueue(2),
            },
            compute2=pp.queues.ProcessQueue(2),
            compute3={
                'compute1': pp.queues.ProcessQueue(2),
                'compute2': pp.queues.ProcessQueue(2),
            },
            printer=pp.queues.ProcessQueue(),
            aggregator=pp.queues.ProcessQueue(),
        )


class Inner(pp.communicators.Flow):

    def __init__(self):
        super().__init__()

        self.compute1 = compute
        self.compute2 = compute

caller = Outer()
caller

compute1-<class '__main__.Inner'>
	compute1-<function compute at 0x7f53d6ca5f80>
	compute2-<function compute at 0x7f53d6ca5f80>
compute2-<function compute at 0x7f53d6ca5f80>
compute3-<class '__main__.Inner'>
	compute1-<function compute at 0x7f53d6ca5f80>
	compute2-<function compute at 0x7f53d6ca5f80>
printer-<function print_pass at 0x7f53d6ca6700>
aggregator-<class 'plasma.parallel_processing.communicators.aggregator.Aggregator'>

In [7]:
with caller:
    for i in range(20):
        caller.put(i)
    
    caller.aggregator.wait()
    print(caller.aggregator.results)

  0%|          | 0/20 [00:00<?, ?it/s]

0.0
1.0
2.0
3.0
4.0
5.0
6.0
7.0
8.0
9.0
10.0
11.0
12.0
13.0
14.0
15.0
16.0
17.0
18.0
19.0
[0, 32, 64, 96, 128, 160, 192, 224, 256, 288, 320, 352, 384, 416, 448, 480, 512, 544, 576, 608]


In [4]:
e = None
try:
    raise Exception('hehe')
except Exception as ex:
    e = ex

print(e)

hehe
